In [7]:
import numpy as np


# 순전파, 역전파 곱셈 계층 구현
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy


apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

dprice = 1

dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


In [11]:
# 순전파 역전파 덧셈계층 구현
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        return x + y

    def backward(self, dout):
        return dout, dout


apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
a = mul_apple_layer.forward(apple, apple_num)
b = mul_orange_layer.forward(orange, orange_num)
c = add_apple_orange_layer.forward(a, b)
d = mul_tax_layer.forward(c, tax)

print(a, b, c, d)

# 역전파
e, dtax = mul_tax_layer.backward(dprice)
f, g = add_apple_orange_layer.backward(e)
h, i = mul_apple_layer.backward(f)
j, k = mul_orange_layer.backward(g)

print(e, dtax, f, g, h, i, j, k)

200 450 650 715.0000000000001
1.1 650 1.1 1.1 2.2 110.00000000000001 3.3000000000000003 165.0


In [13]:
import numpy as np


# Relu
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        return dout


x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

relu = Relu()
a = relu.forward(x)
print(a)

[[ 1.  -0.5]
 [-2.   3. ]]
[[1. 0.]
 [0. 3.]]


In [ ]:
# 시그모이드 계층
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx
